# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [31]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
weather_data_df=pd.read_csv("weatherdata.csv")
weather_data_df


,city name,lat,lon,temp,humidity,cloudiness,wind speed
0,budhlada,29.9333,75.5667,301.09,44,overcast clouds,4.59
1,severo-kurilsk,50.6789,156.1250,283.67,56,clear sky,3.30
2,mataura,-46.1927,168.8643,284.53,98,light rain,3.49
3,upernavik,72.7868,-56.1549,273.29,77,overcast clouds,3.42
4,kungurtug,50.5994,97.5228,283.72,75,overcast clouds,1.59
...,...,...,...,...,...,...,...
562,manzhouli,49.6000,117.4333,288.55,41,overcast clouds,4.17
563,uruguaiana,-29.7547,-57.0883,288.11,67,clear sky,2.06
564,mutis,6.2262,-77.4044,299.08,88,overcast clouds,2.17
565,bamako,12.6500,-8.0000,304.38,26,few clouds,0.40


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [20]:
locations = weather_data_df[["lat", "lon"]].astype(float)
humidity=weather_data_df['humidity'].astype(float)
locations

,lat,lon
0,29.9333,75.5667
1,50.6789,156.1250
2,-46.1927,168.8643
3,72.7868,-56.1549
4,50.5994,97.5228
...,...,...
562,49.6000,117.4333
563,-29.7547,-57.0883
564,6.2262,-77.4044
565,12.6500,-8.0000


In [7]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [17]:
hotel_df=weather_data_df.loc[(weather_data_df['temp']<299) &
                                 (weather_data_df['humidity']<50)
                                 & (weather_data_df['wind speed']<10)]
hotel_df
#clearly something went very wrong in the temp column, but this is for experimental purposes anyway

,city name,lat,lon,temp,humidity,cloudiness,wind speed
10,baker city,44.7749,-117.8344,287.37,35,clear sky,6.17
30,nyurba,63.2842,118.3319,276.93,30,broken clouds,1.49
34,yellowknife,62.4560,-114.3525,276.44,38,scattered clouds,6.17
38,port alfred,-33.5906,26.8910,292.68,37,overcast clouds,5.46
43,fairbanks,64.8378,-147.7164,290.13,31,broken clouds,5.66
...,...,...,...,...,...,...,...
540,opuwo,-18.0607,13.8400,288.76,36,clear sky,1.26
549,asyut,27.1810,31.1837,297.25,33,clear sky,5.47
551,chernyshevskiy,63.0128,112.4714,273.61,46,overcast clouds,1.88
558,fushan,37.4958,121.2581,297.50,42,clear sky,3.47


In [40]:
#hotel_df['Hotel Name']=""
for index, row in locations[:2].iterrows():
    lat=row['lat']
    lon=row['lon']
    #print(location)
    query= f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lon}&radius=5000&keyword=hotel&key={g_key}"
    response=requests.get(query).json()
    print(response)

{'html_attributions': [], 'results': [{'business_status': 'OPERATIONAL', 'geometry': {'location': {'lat': 29.947851, 'lng': 75.5444169}, 'viewport': {'northeast': {'lat': 29.94918052989272, 'lng': 75.54540882989272}, 'southwest': {'lat': 29.94648087010728, 'lng': 75.54270917010729}}}, 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png', 'name': 'Jagat Resort', 'photos': [{'height': 3120, 'html_attributions': ['<a href="https://maps.google.com/maps/contrib/102963406999371089737">Rajan Goyal</a>'], 'photo_reference': 'ATtYBwJckDzSHLQcxC5384LIwwtzz65meXhDdjhYXw_2x5Hn6n4WPJS4lPbIW3LHOAt51DoZybR8A-zINfJJAEX0M4oDvhzDH9Qxj2moiJCITkITjsrMFBkaNc5bconiF5Xin7rvJuUKsB-f8kNfuIJP3-4iW-zOcbwLZWmFyybW7sQz-iAG', 'width': 4160}], 'place_id': 'ChIJv4v_DyQFETkRxlrs1om7_JY', 'plus_code': {'compound_code': 'WGXV+4Q Budhlada R, Punjab, India', 'global_code': '7JXQWGXV+4Q'}, 'rating': 3.9, 'reference': 'ChIJv4v_DyQFETkRxlrs1om7_JY', 'scope': 'GOOGLE', 'types': ['lodging', 'poi

50.6789

In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
